# 两个有序数组的中位数

### 思路

1. 让 A 永远是较短数组，对 A 的切分点 i 做二分（范围 0..len(A)）。
2. half = (len(A) + len(B) + 1) // 2，切分点满足 i + j = half。
3. 对当前 i 算出 Aleft/Aright 与 Bleft/Bright（越界用 ±∞）。
4. 若 Aleft ≤ Bright 且 Bleft ≤ Aright → 切分正确，直接算中位数。
5. 若 Aleft > Bright → i 太大往左；若 Bleft > Aright → i 太小往右。

### 代码

In [5]:
def findMedianSortedArrays(nums1, nums2):
    # 保证 nums1 是较短的
    if len(nums1) > len(nums2):
        nums1, nums2 = nums2, nums1

    m, n = len(nums1), len(nums2)
    total = m + n
    half = (total + 1) // 2

    left, right = 0, m

    from math import inf

    while left <= right:
        i = (left + right) // 2  # 在短数组 nums1 里切
        j = half - i             # 在 nums2 里对应的切分

        # 边界处理（越界时设成 ±∞）
        Aleft  = nums1[i-1] if i > 0 else -inf
        Aright = nums1[i]   if i < m else inf
        Bleft  = nums2[j-1] if j > 0 else -inf
        Bright = nums2[j]   if j < n else inf

        # 📌 打印当前切分状态（Notebook 方便调试）
        print(f"i={i}, j={j}, Aleft={Aleft}, Aright={Aright}, Bleft={Bleft}, Bright={Bright}")

        # ✔ 切分正确
        if Aleft <= Bright and Bleft <= Aright:
            # 总长度是奇数 → 返回左边最大
            if total % 2 == 1:
                result = max(Aleft, Bleft)
                print("\n>>> 中位数 =", result)
                return result
            
            # 总长度是偶数 → 平均值
            left_max = max(Aleft, Bleft)
            right_min = min(Aright, Bright)
            result = (left_max + right_min) / 2
            print("\n>>> 中位数 =", result)
            return result
        
        # ❌ Aleft 太大 → i 取多了 → 向左缩
        elif Aleft > Bright:
            right = i - 1
        
        # ❌ Bleft 太大 → i 取少了 → 向右扩
        else:
            left = i + 1


# ▶ 你可以在 Notebook 里这样运行测试：
print(findMedianSortedArrays([1, 3, 8], [2, 7, 9, 10]))

i=1, j=3, Aleft=1, Aright=3, Bleft=9, Bright=10
i=2, j=2, Aleft=3, Aright=8, Bleft=7, Bright=9

>>> 中位数 = 7
7


### 类似题目（378. 排序矩阵中第k小的元素）

### 思路

1. 在值域区间 [matrix[0][0], matrix[n-1][n-1]] 上做二分（按数值，而不是按下标）。
2. 对每个 mid，用“右上角走法”统计矩阵中 ≤ mid 的数量 count。
3. 右上角走法：matrix[row][col] > mid → 左移；否则 count += col+1 后下移。
4. 若 count >= k → mid 可能太大或刚好，收缩右界；否则 count < k → 左界=mid+1。
5. 当 left == right 时，该值就是矩阵中的第 k 小元素。

### 代码

In [11]:
def kthSmallest(matrix, k):
    n = len(matrix)
    left = matrix[0][0]
    right = matrix[n-1][n-1]

    step = 0  # 用于打印步骤

    while left < right:
        mid = (left + right) // 2
        row = 0
        col = n - 1
        count = 0

        print(f"\n=== 二分 mid = {mid} ===")

        # 右上角走法统计 ≤ mid 的数量
        while row < n and col >= 0:
            step += 1
            print(f"Step {step:>2}: 位置=({row},{col}), 值={matrix[row][col]}")

            if matrix[row][col] > mid:
                print(f"   {matrix[row][col]} > {mid} → 往左走（col--）")
                col -= 1
            else:
                print(f"   {matrix[row][col]} <= {mid} → 本行有 {col+1} 个 ≤ {mid}")
                count += col + 1
                row += 1
                print(f"   累计 count = {count}, 往下走（row++）")

        print(f"本次 mid={mid}，count={count}")

        # 根据 count 调整二分界
        if count >= k:
            print(f"count >= k → right = mid = {mid}")
            right = mid
        else:
            print(f"count < k → left = mid + 1 = {mid+1}")
            left = mid + 1

    print(f"\n🎉 最终答案 = {left}")
    return left


# ▶ 示例运行（你可以换成任意用例）
matrix = [
    [1, 5, 9],
    [10, 11, 13],
    [12, 13, 15]
]

k = 5
kthSmallest(matrix, k)


=== 二分 mid = 8 ===
Step  1: 位置=(0,2), 值=9
   9 > 8 → 往左走（col--）
Step  2: 位置=(0,1), 值=5
   5 <= 8 → 本行有 2 个 ≤ 8
   累计 count = 2, 往下走（row++）
Step  3: 位置=(1,1), 值=11
   11 > 8 → 往左走（col--）
Step  4: 位置=(1,0), 值=10
   10 > 8 → 往左走（col--）
本次 mid=8，count=2
count < k → left = mid + 1 = 9

=== 二分 mid = 12 ===
Step  5: 位置=(0,2), 值=9
   9 <= 12 → 本行有 3 个 ≤ 12
   累计 count = 3, 往下走（row++）
Step  6: 位置=(1,2), 值=13
   13 > 12 → 往左走（col--）
Step  7: 位置=(1,1), 值=11
   11 <= 12 → 本行有 2 个 ≤ 12
   累计 count = 5, 往下走（row++）
Step  8: 位置=(2,1), 值=13
   13 > 12 → 往左走（col--）
Step  9: 位置=(2,0), 值=12
   12 <= 12 → 本行有 1 个 ≤ 12
   累计 count = 6, 往下走（row++）
本次 mid=12，count=6
count >= k → right = mid = 12

=== 二分 mid = 10 ===
Step 10: 位置=(0,2), 值=9
   9 <= 10 → 本行有 3 个 ≤ 10
   累计 count = 3, 往下走（row++）
Step 11: 位置=(1,2), 值=13
   13 > 10 → 往左走（col--）
Step 12: 位置=(1,1), 值=11
   11 > 10 → 往左走（col--）
Step 13: 位置=(1,0), 值=10
   10 <= 10 → 本行有 1 个 ≤ 10
   累计 count = 4, 往下走（row++）
Step 14: 位置=(2,0), 值=12
   12 > 10 → 往左走（col-

11